In [1]:
from keras.models import Sequential, Model
from keras.layers import BatchNormalization
from keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, RMSprop
from keras.utils import to_categorical
import cv2, numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import applications
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import xml.etree.cElementTree as ET
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from shutil import copyfile
from pathlib import Path
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14521686956143902809
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11324823962
locality {
  bus_id: 1
}
incarnation: 14048952974808641988
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


# Test VGG16

In [4]:
nbClasses = 100
img_rows, img_cols, img_channel = 224, 224, 3

base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, img_channel))

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(BatchNormalization())
add_model.add(Dense(512, activation='relu'))
add_model.add(BatchNormalization())
#add_model.add(Dropout(0.5))
add_model.add(Dense(512, activation='relu'))
add_model.add(Dense(nbClasses, activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
for layer in base_model.layers:
    layer.trainable = False
    
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.00002),
              metrics=['accuracy'])

model.summary()

for i, layer in enumerate(base_model.layers):
    print(i, ' ', layer.name)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [5]:
data_path = '/data/test_5classes/'

batch_size = 32
epochs = 12

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    data_path + 'Training/',  
    target_size=(224,224),  
    batch_size=batch_size)  


validation_generator = validation_datagen.flow_from_directory(
        data_path + 'Validation/',
        target_size=(224,224),
        batch_size=batch_size)

test_generator = test_datagen.flow_from_directory(
        data_path + 'Test/',
        target_size=(224,224),
        batch_size=batch_size)

model.fit_generator(
    train_generator,
    steps_per_epoch= train_generator.n // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    callbacks=[ModelCheckpoint(data_path + 'VGG16-transferlearning.model', monitor='val_acc', save_best_only=True)]
)
#model.save_weights(data_path + 'weights/try_VGG16.h5')

for layer in base_model.layers[:14]:
    layer.trainable = False
for layer in base_model.layers[14:]:
    layer.trainable = True 
    
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.00001),
              metrics=['accuracy'])
              
model.fit_generator(
   train_generator,
    steps_per_epoch= train_generator.n // batch_size,
   epochs=4,
   validation_data=test_generator,
   callbacks=[ModelCheckpoint(data_path + 'VGG16-transferlearning.model', monitor='val_acc', save_best_only=True)]
)

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.000001),
            metrics=['accuracy'])
              
model.fit_generator(
   train_generator,
    steps_per_epoch= train_generator.n // batch_size,
   epochs=2,
   validation_data=test_generator,
   callbacks=[ModelCheckpoint(data_path + 'VGG16-transferlearning.model', monitor='val_acc', save_best_only=True)]
)

Found 8870 images belonging to 100 classes.
Found 2658 images belonging to 100 classes.
Found 2221 images belonging to 100 classes.
Epoch 1/12
277/277 [==============================] - 143s 516ms/step - loss: 4.6086 - acc: 0.0403 - val_loss: 4.1854 - val_acc: 0.0937
Epoch 2/12
277/277 [==============================] - 142s 512ms/step - loss: 4.0591 - acc: 0.1029 - val_loss: 3.8696 - val_acc: 0.1418
Epoch 3/12
277/277 [==============================] - 142s 514ms/step - loss: 3.7832 - acc: 0.1426 - val_loss: 3.7227 - val_acc: 0.1630
Epoch 4/12
277/277 [==============================] - 143s 516ms/step - loss: 3.6083 - acc: 0.1661 - val_loss: 3.6063 - val_acc: 0.1905
Epoch 5/12
277/277 [==============================] - 143s 516ms/step - loss: 3.4749 - acc: 0.1958 - val_loss: 3.5125 - val_acc: 0.2152
Epoch 6/12
277/277 [==============================] - 142s 513ms/step - loss: 3.3715 - acc: 0.2102 - val_loss: 3.4418 - val_acc: 0.2238
Epoch 7/12
277/277 [==============================] 

In [6]:
model.save_weights('/data/weights/vgg16_100classes_web.h5')

In [33]:
test_loss, test_acc = model.evaluate_generator(test_generator,109)
print(test_acc)

0.0665137614679
